In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [2]:
img_inv= np.loadtxt("a.txt",delimiter=',',dtype='complex')

In [3]:
um=1e-6
mm=1e-3
nm=1e-9


In [31]:
resabbe=500*nm/(2*0.8)
resabbe

3.125e-07

In [5]:
X,Y=np.shape(img_inv)

In [6]:
w_length=500*nm

In [7]:
k=2*np.pi/w_length

In [8]:
def ondaplana(A,m,n,k,z,zangle,xangle,yangle,deltax,deltay):
    centerx=int(m/2)
    centery=int(n/2)
    
    x=np.arange(-centerx,centerx)
    y=np.arange(-centery,centery)
    
    X1, Y1= np.meshgrid(x,y)
    
    X1=X1*deltax
    Y1=Y1*deltay
    
    wave= A*np.exp(1j*k*(X1*xangle+Y1*yangle+z*zangle))
                   
                   
    return wave
    #iluminacion de la muestra
    
    
    
    
    


In [9]:
ilum=ondaplana(5,720,720,k,5*um,1,0,0,resabbe,resabbe)

In [10]:
ilums=ilum
ilumd=ilum*img_inv
#division de la iluminacion en dos partes interactuante y no interactuante

In [11]:
 Trasn_ilums=np.fft.fftshift(np.fft.fft2(ilums)) 
#transformadas de fresnel por la lente

In [12]:
 Trans_ilumd=np.fft.fftshift(np.fft.fft2(ilumd)) 

In [36]:

tpixel=resabbe/(2*mm*500*nm)
#tpixel=1/(tpixel*720)
tpixel


312.5

In [14]:
tpixel2=tpixel/(200*mm*500*nm)
tpixel2=1/(tpixel2*720)
tpixel2



7.142857142857146e-06

In [15]:
tpixel1=resabbe/(20*mm*500*nm)
tpixel1

71.42857142857144

In [16]:
r1=5.5*mm/tpixel#simensiones del anillo
r1

282.85714285714295

In [17]:
resabbe=500*nm/(2*0.35)
resabbe

7.142857142857144e-07

In [18]:
r2=6.5*mm/(tpixel)
r2

334.2857142857144

In [19]:
def mask_concentric(n,m,r2,r1,aten): #definicion mascara del anillo de fase para la parte interactuante
    mascara=np.zeros((n,m))
    cv.circle(mascara,(int(n/2), int(m/2)),int(r2),1,-1)
    cv.circle(mascara,(int(n/2), int(m/2)),int(r1),0,-1)
    
    mascara2=np.ones((n,m))
    mascara2=mascara2*-1j*aten
    
    mascara=mascara*mascara2
    
    mascara3=np.zeros((n,m))
    cv.circle(mascara3,(int(n/2), int(m/2)),int(r1),1,-1)
    
    mascara=mascara+mascara3
    
    mascara4=np.zeros((n,m))
    cv.circle(mascara4,(int(n/2), int(m/2)),int(r2*1000),1,-1)
    cv.circle(mascara4,(int(n/2), int(m/2)),int(r2),0,-1)
    
    mascara=mascara+mascara4
    
    
    return mascara

In [20]:
%matplotlib qt

In [21]:
T_D=mask_concentric(720,720,r2,r1,0.3)
plt.figure()
plt.imshow(np.abs(T_D)**2)
plt.colorbar()
plt.show()

In [22]:
T_s=np.ones((720,720))*0.3*-1j
#mascara para la parte no interectuante

In [23]:
TFD=T_D*Trans_ilumd

In [24]:
Trans_ilumd

array([[ -8.83275277-30.17769631j,   2.3403401  +8.94319189j,
        -12.23195776+15.69588415j, ...,   6.92775398+28.90065094j,
         -4.76482091 -2.5737073j ,  31.97601999+31.03049826j],
       [-31.1958471 +16.84119727j,   8.34453115-13.44849544j,
         -1.32260585+10.05792494j, ..., -13.3553847 +22.34707978j,
        -19.24848226 -1.23905829j,   6.20495346 +7.3232952j ],
       [-25.22448121 -8.38141462j,  16.28747143 +1.05102397j,
         26.30200494 +7.44163194j, ...,  -3.37732926+11.95059893j,
          7.16378022 -6.51172559j,  10.4062226 -14.96606626j],
       ...,
       [ -3.06297332 -0.92756082j,  24.38861301-10.45435732j,
         19.46978651 +7.36834583j, ...,  -1.76572814+20.93546791j,
         32.12982549 +4.02521853j,  27.94873266 +1.20706409j],
       [  0.67403313 +3.85007698j,   3.10913916-18.75139746j,
        -13.28497962 +0.50394618j, ..., -14.07200896-16.75970897j,
          1.30168711 +6.37943036j, -20.78833175+25.8447284j ],
       [ -0.97247217 -0.2358

In [25]:
TFS=T_s*Trasn_ilums

In [26]:
ES=np.fft.ifft2(TFS)
ED=np.fft.ifft2(TFD)

#trnaformadas inveresa y multiplicacion

In [27]:
ET=ES+ED
#interferencia

In [28]:
plt.figure()
plt.imshow(np.abs(ET)**2)
plt.colorbar()
plt.show()

In [29]:
plt.figure()
plt.imshow(np.abs(img_inv)**2)
plt.colorbar()
plt.show()

In [30]:
plt.figure()
plt.imshow(np.angle(img_inv))
plt.colorbar()
plt.show()